## For Demo

In [92]:
columns=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
         'green_0', 'green_2p', 'green_3p', 'green_4p', 'green_5p', 'green_6', 
         'red_0', 'red_2p', 'red_3p', 'red_4p', 'red_5p', 'red_6', 
         'blue_0', 'blue_2p', 'blue_3p', 'blue_4p', 'blue_5p', 'blue_6', 
         'colours', 'total']

print(len((20, 6, 32, 7, 45, 18, 43, 0, 0, 12, 275, 1255, 12, 1, 6, 13, 251, 2170, 13, 0, 1, 4, 101, 2801, '132', 128)))

26


In [94]:
# cols=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
#          'green_0', 'green_2p', 'green_3p', 'green_4p', 'green_5p', 'green_6', 
#          'red_0', 'red_2p', 'red_3p', 'red_4p', 'red_5p', 'red_6', 
#          'blue_0', 'blue_2p', 'blue_3p', 'blue_4p', 'blue_5p', 'blue_6', 
#          'colours', 'total']
# df = pd.DataFrame(record_list, columns=cols)

# # Save the DataFrame as a CSV file
# df.to_csv('colour_records/rec_10.csv', index=False)

In [108]:
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

service = Service("chrome_driver/chromedriver.exe")
driver = webdriver.Chrome(service=service)

# Create an instance of ActionChains
actions = ActionChains(driver)

driver.get("https://logigames.bet9ja.com/Games/Launcher?gameId=11000&provider=0&pff=1&skin=201")
# driver.maximize_window()
time.sleep(5)

record_list = []

while True:
    counter = driver.find_element(By.XPATH, "//div[@class='timeline__value-txt']").text
    # print(counter)

    if int(counter) < 44 and int(counter) > 20:

        driver.find_element(By.XPATH, "//div[@class='stats__toggle ']").click()
        time.sleep(0.5)
        driver.find_element(By.XPATH, "//a[contains(text(),'Rainbow')]").click()
        driver.find_element(By.XPATH, "//div[@class='dd__trigger active dd__menu-open']//div[@class='dd__menu-item--value'][normalize-space()='All time']").click()
        time.sleep(3)

        stats = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text
        stats_record = tuple([int(x.split(" ")[0].split("\n")[1]) for x in stats.split("%")[1:]])

        driver.find_element(By.XPATH, "//a[normalize-space()='History']").click()
        time.sleep(2)
        
        history = driver.find_element(By.XPATH, "//div[@class='p-stats__content']").text
        colours_count = history.split(" ")[9:][0].split("\n")[-1]
        sum_colours = int(history.split(" ")[9:][1].split("\n")[0])

        ball_path = "/html[1]/body[1]/div[1]/div[1]/div[1]/div[1]/main[1]/div[4]/div[2]/table[1]/tbody[1]/tr[1]/td[2]/div[1]/span["
        ball_1 = int(driver.find_element(By.XPATH, f"{ball_path}1]").text)
        ball_2 = int(driver.find_element(By.XPATH, f"{ball_path}2]").text)
        ball_3 = int(driver.find_element(By.XPATH, f"{ball_path}3]").text)
        ball_4 = int(driver.find_element(By.XPATH, f"{ball_path}4]").text)
        ball_5 = int(driver.find_element(By.XPATH, f"{ball_path}5]").text)
        ball_6 = int(driver.find_element(By.XPATH, f"{ball_path}6]").text)
        
        all_balls = (ball_1, ball_2, ball_3, ball_4, ball_5, ball_6)
        all_record = all_balls + stats_record + (colours_count, sum_colours)
        record_list.append(all_record)
        print(all_record)


        while True:
            counter = driver.find_element(By.XPATH, "//div[@class='timeline__value-txt']").text
            # print(counter)
            if int(counter) <= 42 and int(counter) >= 39:
                # print("waiting for new records")

                # Trading
                driver.find_element(By.XPATH, "//div[@class='stats__toggle active']").click()
                # time.sleep(5)
                break
            else:
                time.sleep(1)

    else:
        time.sleep(1)

# Close the browser session
driver.quit()

(14, 15, 10, 29, 13, 24, 4, 0, 2, 2, 20, 5632, 19, 0, 6, 18, 56, 1410, 14, 0, 1, 4, 155, 395, '222', 105)


In [107]:
cols=['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 
         'green_0', 'green_2p', 'green_3p', 'green_4p', 'green_5p', 'green_6', 
         'red_0', 'red_2p', 'red_3p', 'red_4p', 'red_5p', 'red_6', 
         'blue_0', 'blue_2p', 'blue_3p', 'blue_4p', 'blue_5p', 'blue_6', 
         'colours', 'total']
df = pd.DataFrame(record_list, columns=cols)

# Save the DataFrame as a CSV file
df.to_csv('colour_records/rec_7.csv', index=False)

In [95]:
df[df.duplicated()]

,b1,b2,b3,b4,b5,b6,green_0,green_2p,green_3p,green_4p,...,red_5p,red_6,blue_0,blue_2p,blue_3p,blue_4p,blue_5p,blue_6,colours,total


## For Real Trading

In [ ]:
import pandas as pd
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time

service = Service("chrome_driver/chromedriver.exe")
driver = webdriver.Chrome(service=service)

driver.get("https://casino.bet9ja.com/casino/category/popular")
# driver.maximize_window()
time.sleep(5)

# Create an instance of ActionChains
actions = ActionChains(driver)

# login 
username = "Nnabueze"
password = "cjval1610"
driver.find_element(By.XPATH, "//div[@title='Log In']").click()
time.sleep(1)
driver.find_element(By.XPATH, "//input[@id='01']").send_keys(username)
driver.find_element(By.XPATH, "//input[@id='02']").send_keys(password)
driver.find_element(By.XPATH, "//div[@class='btn-primary-l mt20']").click()
time.sleep(3)

element_to_hover = driver.find_element(By.XPATH, "//div[@id='11000']//div[@class='game__info']")
actions.move_to_element(element_to_hover).perform()

# # for demo training
# driver.find_element(By.XPATH, "//div[@id='11000']//button[@title='Demo'][normalize-space()='Demo']").click()

# for real trading
driver.find_element(By.XPATH, "//div[@id='11000']//div[@class='game__info']//button[@title='Play Now'][normalize-space()='Play Now']").click()

time.sleep(5)

# Get the handles of all open windows
window_handles = driver.window_handles

# Switch to the new window
new_window_handle = window_handles[-1]  # Get the handle of the last window in the list
driver.switch_to.window(new_window_handle)
# print(len(window_handles))

# Now you are in the new window

# Continue with actions or assertions in the new window
driver.maximize_window()
time.sleep(1)
driver.find_element(By.XPATH, "//div[@class='games']//a[1]").click()

# # After finishing with the new window, you can switch back to the original window if needed
# driver.switch_to.window(window_handles[0])
# driver.maximize_window()

# # Close the browser session
# driver.quit()